In [1]:
import os
import pandas as pd
import numpy as np
import datetime
from datetime import time

In [2]:
renfe = pd.read_csv("../Data/renfe.csv")

### Renaming columns

In [3]:
renfe = renfe.rename(columns={"insert_date":"price_check_datetime", "train_class":"ticket_class",\
                              "start_date":"departure_datetime", "end_date":"arrival_datetime", "origin":"Origin",\
                              "destination":"Destination"})

### City codes for origin and destination

In [4]:
renfe.loc[renfe["Origin"].str.startswith("M"), "origin"] = "MAD"
renfe.loc[renfe["Origin"].str.startswith("B"), "origin"] = "BAR"
renfe.loc[renfe["Origin"].str.startswith("V"), "origin"] = "VAL"
renfe.loc[renfe["Origin"].str.startswith("S"), "origin"] = "SEV"
renfe.loc[renfe["Origin"].str.startswith("P"), "origin"] = "PON"

renfe.loc[renfe["Destination"].str.startswith("M"), "destination"] = "MAD"
renfe.loc[renfe["Destination"].str.startswith("B"), "destination"] = "BAR"
renfe.loc[renfe["Destination"].str.startswith("V"), "destination"] = "VAL"
renfe.loc[renfe["Destination"].str.startswith("S"), "destination"] = "SEV"
renfe.loc[renfe["Destination"].str.startswith("P"), "destination"] = "PON"

### Creating new columns

In [5]:
renfe["route"] = renfe[["origin", "destination"]].apply(lambda x: "-".join(x), axis=1)
renfe["Abb_departure_date"] = renfe["departure_datetime"].apply(lambda x: x[5:10])
renfe["abb_departure_time"] = renfe["departure_datetime"].apply(lambda x: x[11:16])
renfe["abb_departure_date"] = renfe["Abb_departure_date"].replace(r"(\d+)-(\d+)", r"\2-\1", regex=True)
renfe["semi_identifier"] = renfe[["abb_departure_date", "abb_departure_time"]].apply(lambda x: "_".join(x), axis=1)

### Datetime column converted from string to a datetime format and placed separately

In [6]:
renfe["price_check_datetime"] = pd.to_datetime(renfe["price_check_datetime"])
renfe["departure_datetime"] = pd.to_datetime(renfe["departure_datetime"])

In [7]:
renfe["departure_date"] = renfe.departure_datetime.dt.date
renfe["departure_time"] = renfe.departure_datetime.dt.time
renfe["price_check_date"] = renfe.price_check_datetime.dt.date
renfe["price_check_time"] = renfe.price_check_datetime.dt.time

### Creating additional columns now that date & time columns are formatted properly

In [8]:
day_of_week={0:"Monday", 1:"Tuesday", 2:"Wednesday", 3:"Thursday", 4:"Friday", 5:"Saturday", 6:"Sunday"}

renfe["departure_weekday"] = renfe["departure_datetime"].dt.dayofweek.map(day_of_week)
renfe["price_check_weekday"] = renfe["price_check_datetime"].dt.dayofweek.map(day_of_week)
renfe["days_to_departure"] = renfe["departure_date"] - renfe["price_check_date"]

### Dropping unnecessary column

In [9]:
renfe.drop("Unnamed: 0", axis=1, inplace=True)
renfe.drop("Origin", axis=1, inplace=True)
renfe.drop("Destination", axis=1, inplace=True)
renfe.drop("arrival_datetime", axis=1, inplace=True)
renfe.drop("price_check_datetime", axis=1, inplace=True)
renfe.drop("departure_datetime", axis=1, inplace=True)
renfe.drop("Abb_departure_date", axis=1, inplace=True)

### Changing order of columns

In [10]:
renfe.columns

Index(['train_type', 'price', 'ticket_class', 'fare', 'origin', 'destination',
       'route', 'abb_departure_time', 'abb_departure_date', 'semi_identifier',
       'departure_date', 'departure_time', 'price_check_date',
       'price_check_time', 'departure_weekday', 'price_check_weekday',
       'days_to_departure'],
      dtype='object')

In [11]:
column_order = ['origin', 'destination', 'departure_date', 'departure_time', 'price', 'price_check_date',
                'price_check_time', 'train_type', 'ticket_class', 'fare', 'route', 'abb_departure_date',
                'abb_departure_time', 'semi_identifier', 'departure_weekday', 'price_check_weekday',
                'days_to_departure']

renfe = renfe[column_order]

In [12]:
renfe.head(3)

,origin,destination,departure_date,departure_time,price,price_check_date,price_check_time,train_type,ticket_class,fare,route,abb_departure_date,abb_departure_time,semi_identifier,departure_weekday,price_check_weekday,days_to_departure
0,MAD,SEV,2019-05-29,06:20:00,38.55,2019-04-19,05:31:43,AV City,Turista,Promo,MAD-SEV,29-05,06:20,29-05_06:20,Wednesday,Friday,40 days
1,MAD,SEV,2019-05-29,07:00:00,53.40,2019-04-19,05:31:43,AVE,Turista,Promo,MAD-SEV,29-05,07:00,29-05_07:00,Wednesday,Friday,40 days
2,MAD,SEV,2019-05-29,07:30:00,47.30,2019-04-19,05:31:43,AVE,Turista,Promo,MAD-SEV,29-05,07:30,29-05_07:30,Wednesday,Friday,40 days


### dropping invalid rows

In [13]:
renfe = renfe[(renfe["price_check_date"] <= renfe["departure_date"])]

In [14]:
renfe.isna().sum()

origin                      0
destination                 0
departure_date              0
departure_time              0
price                  309982
price_check_date            0
price_check_time            0
train_type                  0
ticket_class             9592
fare                     9592
route                       0
abb_departure_date          0
abb_departure_time          0
semi_identifier             0
departure_weekday           0
price_check_weekday         0
days_to_departure           0
dtype: int64

We are going to remove the ones with ticket_class and fare unavailable, since we can't discretionally replace them
and we will need values in that variable down the road. Let's leave the NaN values of price for now, since most of them are legit because of the way the web-scrapping was done in the frame of how the dataset is structured.

In [15]:
nulli = renfe[renfe["fare"].isna()].index

In [16]:
renfe = renfe.drop(nulli, axis=0)

Creating the same dataframe but just for April departing trains

In [17]:
limit_1 = datetime.datetime.strptime('2019-04-01', '%Y-%m-%d').date()

In [18]:
limit_2 = datetime.datetime.strptime('2019-05-01', '%Y-%m-%d').date()

In [19]:
renfe_apr = renfe[(renfe["departure_date"] < limit_2) & (renfe["departure_date"] >= limit_1)]

In [20]:
renfe_apr.head(3)

,origin,destination,departure_date,departure_time,price,price_check_date,price_check_time,train_type,ticket_class,fare,route,abb_departure_date,abb_departure_time,semi_identifier,departure_weekday,price_check_weekday,days_to_departure
24,MAD,SEV,2019-04-23,06:20:00,43.55,2019-04-19,05:31:54,AV City,Turista,Promo,MAD-SEV,23-04,06:20,23-04_06:20,Tuesday,Friday,4 days
25,MAD,SEV,2019-04-23,07:00:00,76.30,2019-04-19,05:31:54,AVE,Turista,Flexible,MAD-SEV,23-04,07:00,23-04_07:00,Tuesday,Friday,4 days
26,MAD,SEV,2019-04-23,07:30:00,76.30,2019-04-19,05:31:54,AVE,Turista,Flexible,MAD-SEV,23-04,07:30,23-04_07:30,Tuesday,Friday,4 days


### Saving to csv. First version of the renfe data cleant

In [21]:
renfe.to_csv("../Data/renfe_clean_1.csv", index=False)
renfe_apr.to_csv("../Data/renfe_april_clean_1.csv", index=False)